# Extending Data Model

**Prerequisite**:

- Access to a CDF Project.
- Know how to install and setup Python.
- Launch a Python notebook.

In this tutorial, we will show you how to extend a data model using the `CogniteCore` model as an example.


## Creating the Base Model

The `CogniteCore` model is an immutable model versioned by `Cognite`. To extend this model we first create a copy of it
in our own space, such that we can start to do modifications to it.

In [41]:
from cognite.neat import NeatSession, get_cognite_client

In [44]:
client = get_cognite_client(".env")

Found .env file in repository root. Loaded variables from .env file.


In [45]:
neat = NeatSession(client, verbose=True)

neat.read.cdf.data_model(("cdf_cdm", "CogniteCore", "v1"))

CDF <class 'tuple'> ('cdf_cdm', 'CogniteCore', 'v1') read successfully


[]

We have now downloaded the `CogniteCore` model, and need to run `Neat`'s verification on it

In [49]:
issues = neat.verify()

In [52]:
print(issues[0].as_message())

    to by view property PropertyId(source=ViewId(space='cdf_cdm', external_id='Cognite3DObject', version='v1'), property='asset') does not exist. This will be ignored.
    
Fix: Ensure the view ViewId(space='cdf_cdm', external_id='CogniteAsset', version='v1') has a object3D property


The `CogniteCore` model consists of 33 views, including views for 3D, Annotation, as well as several views that are used as interfaces.

We simpliy the core model by dropping these views.

In [47]:
neat.prepare.data_model.reduce(drop=["3D", "Annotation", "BaseViews"])

Next we will turn the `CogniteCore` model into the basis for our new model `ExtensionCore`

In [48]:
neat.prepare.data_model.to_extension(new_data_model_id=("sp_doctrino_space", "ExtensionCore", "v1"), prefix="MyOrg")

The new base model is now ready. We can 

In [13]:
session.to.cdf.data_model()

,name,created
0,spaces,1.0
1,containers,NaN
2,views,12.0
3,data_models,1.0


In [14]:
from cognite.neat._utils.cdf.loaders import SpaceLoader

In [15]:
SpaceLoader(client).clean("sp_anders_space")

Deleted 12 views
Deleted 1 data models
Deleted space ['sp_anders_space']
